# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109A Introduction to Data Science: 

## Final Project Group 40: Trump Tweets and Market Volatility


**Harvard University**<br/>
**Fall 2019**<br/>
**Group**: Dylan DeMarco, Daiki Ina, Roger Zhang, Jerry Huang

<hr style="height:2pt">

In [47]:
import requests
from IPython.core.display import HTML, display
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

import seaborn as sns
sns.set(style='whitegrid')
pd.set_option('display.width', 1500)
pd.set_option('display.max_columns', 100)

import pandas_profiling

In [55]:
vix_df = pd.read_csv('./data/raw/vix_data.csv', index_col=0)
vix_df = vix_df.drop(columns=['VIX High', 'VIX Low', 'VIX Close'])
vix_df.head(3)

,Date,VIX Open
3238,2016-11-09,20.70
3239,2016-11-10,14.01
3240,2016-11-11,14.83


In [56]:
tweet_df = pd.read_csv('./data/raw/tweet_data.csv', index_col=0)
tweet_df.head(3)
tweet_count = tweet_df.group_by('day').count()
tweet_daily = tweet_df.group_by('day').sum()

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
0,Twitter for Android,Such a beautiful and important evening! The fo...,11-09-2016 11:36:58,220796,633253,False,796315640307060738
1,Twitter for iPhone,Happy 241st birthday to the U.S. Marine Corps!...,11-10-2016 19:31:27,45576,169729,False,796797436752707585
2,Twitter for Android,A fantastic day in D.C. Met with President Oba...,11-11-2016 02:10:46,37788,192638,False,796897928048766976


In [57]:
# dow_df = pd.read_csv('.data/cleaned/dow_data.csv')
# dow_df.head(3)

### Base Model

### Num Tweets

### Num Tweets (Filtered)

### Granular Tweet

### Granular Tweet (Filtered)

### Granular Tweet + DOW